<a href="https://colab.research.google.com/github/CShorten/AIWeeklyUpdates/blob/main/Investigating_Generalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Utilizing AugmentationZoo, TensorFlow Addons, and Imgaug </h1>

In [ ]:
!git clone https://github.com/CShorten/AugmentationZoo.git
import os
os.chdir("AugmentationZoo")

In [ ]:
!pip install tensorflow-addons

In [ ]:
!pip install --upgrade imgaug

<h1>
Testing Inductive Bias in Computer Vision </h1>
<p>
In Computer Vision, <b>Global</b> and <b>Local</b> priors bias the processing of an image data structure.</p>
<ul>
  <li> <b>Local</b> priors focus on smaller windows of the image, such as contiguous 7 x 7 pixel squares. </li>
  <li><b>Global</b> priors attend on the entire 256 x 256 image at once.</li>
</ul>

<h1> Architecture Bias </h1>
  <ul>
    <li>The Convolutional Network has a strong <b>Local</b> bias.</li>
    <li>The Vision Transformer has a strong <b>Global</b> bias.</li>
  </ul>


In [3]:
from Datasets import get_cifar_10
x_train, y_train, x_test, y_test = get_cifar_10()

from Models import ResNet50_with_upsampling, create_vit_classifier, compile_model
ResNet50 = ResNet50_with_upsampling(x_train)
ViT = create_vit_classifier(x_train)
compile_model(ResNet50)
compile_model(ViT) # merge into one line of code

<h1> Data Augmentation Bias </h1>

Similarly, data augmentations contain <b> Global </b> and <b> Local </b> priors.
<ul>
  <li> The crop augmentation 
  <li> The rotate augmentation

In [ ]:
from imgaug import augmenters as iaa
rotate = iaa.Affine(rotate=(-45,45))
crop = iaa.Crop(percent=(0, 0.2))

<h1> There are no Inductive Biases in the Models with Random Weights </h1>

In [11]:
from AugEval import model_list_aug_results
train_results, test_results = model_list_aug_results(
    [ResNet50, ViT], ["ResNet50", "ViT"],
    [rotate, crop], ["Rotate", "Crop"],
    x_train, y_train, x_test, y_test
)

from Checkpoints import save_file
save_file(train_results, "train_results.csv")
import pandas as pd
df = pd.read_csv("train_results.csv")
df.head()

,Model Name,Original,Rotate,Crop
0,ResNet50,0.09988,0.1000,0.09996
1,ViT,0.08654,0.0884,0.08510


<h1> Training the Models <h1>
<ul>
  <li> Each model takes about 70 seconds per epoch. </li>
  <li> Each model is trained for 100 epochs. </li>
  <li> The total training time is ~120 minutes / 2 hours </li>
</ul>

Epoch count is a potential confounder that will be isolated in future work.

In [13]:
ResNet50.fit(x_train, y_train, batch_size=256, epochs=100)

Epoch 1/100
196/196 [==============================] - 81s 367ms/step - loss: 3.1735 - accuracy: 0.2251
Epoch 2/100
196/196 [==============================] - 71s 364ms/step - loss: 1.4773 - accuracy: 0.4574
Epoch 3/100
196/196 [==============================] - 72s 365ms/step - loss: 1.2473 - accuracy: 0.5448
Epoch 4/100
196/196 [==============================] - 72s 366ms/step - loss: 1.0707 - accuracy: 0.6150
Epoch 5/100
196/196 [==============================] - 72s 366ms/step - loss: 0.9432 - accuracy: 0.6613
Epoch 6/100
196/196 [==============================] - 72s 366ms/step - loss: 0.8121 - accuracy: 0.7094
Epoch 7/100
196/196 [==============================] - 72s 366ms/step - loss: 0.6912 - accuracy: 0.7530
Epoch 8/100
196/196 [==============================] - 72s 366ms/step - loss: 0.5789 - accuracy: 0.7953
Epoch 9/100
196/196 [==============================] - 72s 366ms/step - loss: 0.4882 - accuracy: 0.8266
Epoch 10/100
196/196 [==============================] - 72s 366m

In [12]:
ViT.fit(x_train, y_train, batch_size=256, epochs=100)

Epoch 1/100
196/196 [==============================] - 78s 356ms/step - loss: 1.9998 - accuracy: 0.3348
Epoch 2/100
196/196 [==============================] - 70s 357ms/step - loss: 1.4570 - accuracy: 0.4752
Epoch 3/100
196/196 [==============================] - 70s 356ms/step - loss: 1.2755 - accuracy: 0.5403
Epoch 4/100
196/196 [==============================] - 70s 356ms/step - loss: 1.1545 - accuracy: 0.5889
Epoch 5/100
196/196 [==============================] - 70s 357ms/step - loss: 1.0478 - accuracy: 0.6289
Epoch 6/100
196/196 [==============================] - 70s 357ms/step - loss: 0.9718 - accuracy: 0.6573
Epoch 7/100
196/196 [==============================] - 70s 357ms/step - loss: 0.8930 - accuracy: 0.6839
Epoch 8/100
196/196 [==============================] - 70s 357ms/step - loss: 0.8291 - accuracy: 0.7062
Epoch 9/100
196/196 [==============================] - 70s 356ms/step - loss: 0.7619 - accuracy: 0.7277
Epoch 10/100
196/196 [==============================] - 70s 356m

In [ ]:
from Checkpoints import save_file
train_results, test_results = model_list_aug_results(
    [ResNet50, ViT], ["ResNet50-NoAug", "ViT-NoAug"],
    [rotate, crop], ["Rotate", "Crop"],
    x_train, y_train, x_test, y_test
)
save_file(train_results, "train_results.csv")
save_file(test_results, "test_results.csv")

<h1> Commentary on Results </h1>

<p> Both the ResNet50 and 20M parameter ViT are overfitted, see original train accuracy above. The ViT was quicker to overfit the training data. This illustrates the capacity of transformers to fit their training data. This is further evidenced by their appeal in self-supervised learning. In self-supervised learning, the dataset are very large and the model usually never repeats data during training. In this setting, the capacity of transformers is useful, but in smaller setings the strong inductive bias of convolutions is less prone to overfitting.</p>

<h1> References </h1>
<h6> Language Models are Few-Shot Learners. Brown et al. 2021.</h6>
<h6>A Farewell to the Bias-Variance Tradeoff?
An Overview of the Theory of Overparameterized Machine Learning. Yehuda Dar et al. 2021.</h6>

In [17]:
import pandas as pd
df = pd.read_csv("train_results.csv")
df.head()

,Model Name,Original,Rotate,Crop
0,ResNet50-NoAug,0.95392,0.47966,0.61640
1,ViT-NoAug,1.00000,0.57608,0.72864


In [19]:
df = pd.read_csv("test_results.csv")
df.head()

,Model Name,Original,Rotate,Crop
0,ResNet50-NoAug,0.7221,0.4290,0.5545
1,ViT-NoAug,0.7578,0.5072,0.6508


<h1> Please Cite </h1>
<h2> Investigating the Generalization of Image Classifiers with Augmented Test Sets. Shorten and Khoshgoftaar, In ICTAI 2021. </h2>